<a href="https://colab.research.google.com/github/ice4869/titanic-analysis/blob/main/%E9%90%B5%E9%81%94%E5%B0%BC%E8%99%9F%E5%AD%98%E6%B4%BB%E9%A0%90%E6%B8%AC%E5%B0%88%E9%A1%8C%E5%AF%A6%E4%BD%9C%EF%BC%882%EF%BC%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

鐵達尼號存活預測專題實作（2）

1. 在資料分析的工作流程中，在不考慮準確度情況下有哪些必備的環節？

在資料分析的工作流程中，不考慮準確度的情況下，以下是一些必備的環節：

了解問題和目標：明確了解分析的問題陳述和目標。這有助於定義分析的範圍和方向。
整理和收集資料：確保資料可用且符合分析需求。這包括收集、整理、清理和轉換資料，以確保資料品質和一致性。

探索性資料分析（EDA）：對資料進行初步的探索性分析，以了解資料的特徵、分布、關聯性等。這有助於發現任何異常值、缺失值或資料偏差，並提供對後續分析的洞察。

特徵選擇和工程：根據分析目標，選擇相關的特徵並進行必要的轉換或衍生。這包括對類別特徵進行編碼、處理缺失值、進行標準化或正規化等。

建立模型和分析：根據分析目標和資料特性，選擇合適的分析模型或方法。這可能包括機器學習模型、統計模型、規則引擎等。進行模型建立、訓練和評估，並從中獲得洞察。

結果呈現和解讀：將分析結果以可視化和易於理解的形式呈現出來。這可以包括報告、圖表、圖形或交互式儀表板。解釋和解讀結果，並提供相關的洞察和建議。

2. 接下來請在不考慮準確度的前提下完成 Baseline ，讓已讀入的資料完成必要的資料前處理的操作。

（補充：將 df 變成乾淨的 df_train，讓原始資料變成是模型跑得動的資料格式。）


In [4]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

#讀取資料集
df = pd.read_csv('https://raw.githubusercontent.com/dsindy/kaggle-titanic/master/data/train.csv')

#處理缺失值
imputer = SimpleImputer(strategy = "mean")
df['Age'] = imputer.fit_transform(df[['Age']])

#選擇需要的欄位
selected_features = ['Pclass' , 'Sex' , 'Age' , 'SibSp' , 'Parch' , 'Fare' , 'Embarked']
df_selected = df[selected_features + ['Survived'] ]

#偏碼類別欄位
encoder = OneHotEncoder(drop = 'first')
transformer = ColumnTransformer(transformers = [('encoder' , encoder , [1 , 6] )] , remainder = 'passthrough')
df_encoded = transformer.fit_transform(df_selected)

#建立模型使用的資料集
df_train = pd.DataFrame(df_encoded , columns = ['Female' , 'Embarked_Q' , 'Embarked_S'] + selected_features)
df_train['Survived'] = df_selected['Survived']

#確認處理後的資料
print(df_train.head())

#建立模型
columns_X = set(df_train.columns) - {'Survived'}
columns_y = ['Survived']


train_X = df_train[columns_X]
train_y = df_train[columns_y]


from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score


log = LogisticRegression(random_state=0, max_iter=3000)
scores = cross_val_score(log, train_X, train_y.values.ravel(),cv=5,scoring='accuracy')
print(scores)

   Female  Embarked_Q  Embarked_S  Pclass  Sex   Age  SibSp  Parch     Fare  \
0     1.0         0.0         1.0     0.0  3.0  22.0    1.0    0.0   7.2500   
1     0.0         0.0         0.0     0.0  1.0  38.0    1.0    0.0  71.2833   
2     0.0         0.0         1.0     0.0  3.0  26.0    0.0    0.0   7.9250   
3     0.0         0.0         1.0     0.0  1.0  35.0    1.0    0.0  53.1000   
4     1.0         0.0         1.0     0.0  3.0  35.0    0.0    0.0   8.0500   

   Embarked  Survived  
0       0.0         0  
1       1.0         1  
2       1.0         1  
3       1.0         1  
4       0.0         0  


<ipython-input-4-cfe3c4a0e4b8>:35: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  train_X = df_train[columns_X]


[1. 1. 1. 1. 1.]
